In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Step 1: Import and Preprocessing
# Importing the dataset
dataset = pd.read_csv('data/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [3]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
X.nunique()

CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
dtype: int64

In [5]:
X.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [6]:
one_hot_columns = [col for col in X.columns if col not in  ['CreditScore','Age','Balance', 'EstimatedSalary']]

In [7]:
X = pd.get_dummies(X, columns = one_hot_columns)

In [8]:
# Convert Ag in buckets
X['Age'] = pd.cut(X['Age'], 10)

In [9]:
X = pd.get_dummies(X,columns = ['Age'])

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X),columns = X.columns)

In [11]:
# Feature Scaling
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [26]:
# There are two ways of initializing a neiral network:
# 1. by defining the sequence of layers
# 2. by defining the graph

# In this problem, we will intialize it by defining the seqquence of layers

# Step 2: Import libraries
import keras
from keras.models import Sequential
from keras.layers import Dense

# Step 3: Initialize
classifier_initial = Sequential()

# Step 4: Add layers

# Input layer (designated by input_dim = 11) and first hidden layer
classifier_initial.add(Dense(units = 19 , kernel_initializer = 'uniform', activation = 'relu', input_dim = 37))
# second hidden layer
classifier_initial.add(Dense(units = 19 , kernel_initializer = 'uniform', activation = 'relu'))
# output layer
classifier_initial.add(Dense(units = 1 , kernel_initializer = 'uniform', activation = 'sigmoid'))

# Step 5: compile ann - apply stochastic gradient descent
classifier_initial.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Step 6: Fit the model:
classifier_initial.fit(X_train,y_train, batch_size = 10, epochs = 10)

Epoch 1/10
7500/7500 [==============================] - 2s 263us/step - loss: 0.4189 - accuracy: 0.8003
Epoch 2/10
7500/7500 [==============================] - 1s 199us/step - loss: 0.3759 - accuracy: 0.8479
Epoch 3/10
7500/7500 [==============================] - 2s 203us/step - loss: 0.3680 - accuracy: 0.8556
Epoch 4/10
7500/7500 [==============================] - 1s 197us/step - loss: 0.3618 - accuracy: 0.8588
Epoch 5/10
7500/7500 [==============================] - 1s 196us/step - loss: 0.3575 - accuracy: 0.8605
Epoch 6/10
7500/7500 [==============================] - 2s 223us/step - loss: 0.3535 - accuracy: 0.8612
Epoch 7/10
7500/7500 [==============================] - 2s 219us/step - loss: 0.3497 - accuracy: 0.8612
Epoch 8/10
7500/7500 [==============================] - 2s 217us/step - loss: 0.3465 - accuracy: 0.8632
Epoch 9/10
7500/7500 [==============================] - 1s 198us/step - loss: 0.3439 - accuracy: 0.8644
Epoch 10/10
7500/7500 [==============================] - 1s 192u

In [27]:
# shape of weights
for elements in classifier_initial.get_weights():
    print(elements.shape)

(37, 19)
(19,)
(19, 19)
(19,)
(19, 1)
(1,)


In [28]:
# Step 7: Make Predictions
y_pred = classifier_initial.predict(X_test)
# y_pred is probabolity --> convert it into class prediction to get y_pred_2 
y_pred_2 = pd.DataFrame(y_pred).apply(lambda row: 1 if row[0]>0.5 else 0, axis = 1)

In [29]:
# Step 8: Accuracy
import sklearn.metrics as metrics
print(f'accuracy: {metrics.accuracy_score(y_test, y_pred_2)}')
print(f'f1_score: {metrics.f1_score(y_test, y_pred_2)}')
print(f'precision: {metrics.precision_score(y_test, y_pred_2)}')
print(f'recall: {metrics.recall_score(y_test, y_pred_2)}')

accuracy: 0.8436
f1_score: 0.535077288941736
precision: 0.677710843373494
recall: 0.44204322200392926


# Step 9: Evaluating the ANN:
Judging our models's performance on one accuracy and one test set is not the best way to evaluate the model. Changing the test set will change the accuracy of our model slightly. To curb this issue, we will use k-fold cross validation.

!['cv'](images/cv.png)

### Cross validation steps:
1. Train on k-1 folds
2. Test on remaining one
3. Take mean of all k accuracies
4. Find standard deviation of all accuracies
5. Based on accuracy and standard deviations, we can see which of the following cases our model satisfies:

!['bias_variance'](images/bias_variance.JPG)

### Implementation steps:

1. cross_val_score is sklearn function
2. create a keras wrapper for sklearn so that the keras classifier can be used in sklearn cross_val_score
3. 

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [17]:
# defining the function to be passed to KerasClassifier to convert it to sklearn classifier
# only define the nn architecture. training and testing will be done by cross_val_Score
def nn_architecture():
    #initialize
    classifier = Sequential()
    # add input and first hidden layer
    classifier.add(Dense(units = 19, activation = 'relu', kernel_initializer= 'uniform' , input_dim =  37))
    # add second hidden layer
    classifier.add(Dense(units = 19, activation = 'relu', kernel_initializer= 'uniform' ))
    # add output layer
    classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'uniform'))
    # compile
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [18]:
classifier_2 = KerasClassifier(build_fn= nn_architecture, batch_size = 10, nb_epoch = 100)

In [19]:
accuracies = cross_val_score(estimator = classifier_2,X = X_train, y =  y_train, cv = 10, n_jobs=-1)

In [20]:
avg_accuracy = accuracies.mean()
std_accuracy = accuracies.std()

print(f'accuracy = {avg_accuracy*100:.2f}% +/- {std_accuracy*100:.2f}%')

accuracy = 84.77% +/- 1.35%


# Step 10: Improving Accuracy
1. Dropout Regularisation
2. Hyper parametric tuning

In [39]:
from keras.layers import Dropout
#nn architecture with dropouts
def nn_architecture_2():
    classifier = Sequential()
    # add layers with dropout regularization
    classifier.add(Dense(units = 19, activation = 'relu', kernel_initializer = 'uniform', input_dim = 37))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 19, activation = 'relu', kernel_initializer = 'uniform'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'uniform'))

    # compile
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return classifier
# cross validation
classifier_wrapper = KerasClassifier(build_fn = nn_architecture_2, batch_size = 10, nb_epoch = 100)
accuracies_dropout = cross_val_score(estimator=classifier_wrapper, X = X_train, y = y_train, cv = 10)

Epoch 1/1
750/750 [==============================] - 0s 155us/step
Epoch 1/1
750/750 [==============================] - 0s 165us/step
Epoch 1/1
750/750 [==============================] - 0s 221us/step
Epoch 1/1
750/750 [==============================] - 0s 160us/step
Epoch 1/1
750/750 [==============================] - 0s 157us/step
Epoch 1/1
750/750 [==============================] - 0s 148us/step
Epoch 1/1
750/750 [==============================] - 0s 144us/step
Epoch 1/1
750/750 [==============================] - 0s 153us/step
Epoch 1/1
750/750 [==============================] - 0s 209us/step
Epoch 1/1
750/750 [==============================] - 0s 145us/step


In [40]:
# print(f'training set initital accuracy: {metrics.accuracy_score(y_train, classifier_initial(X_train))}')
print(f'test set initital accuracy: {metrics.accuracy_score(y_test, y_pred_2)}')
print(f'cross validation accuracy: {avg_accuracy}')
print(f'training set accuracy after dropout regularisation: {accuracies_dropout.mean()}')

test set initital accuracy: 0.8436
cross validation accuracy: 0.8477333307266235
training set accuracy after dropout regularisation: 0.8322666704654693


In [41]:
accuracies.mean()

0.8477333307266235

### Hyperparametric tuning

In [55]:
def nn_architecture_3(optimizer):
    classifier = Sequential()
    # add layers with dropout regularization
    classifier.add(Dense(units = 19, activation = 'relu', kernel_initializer = 'uniform', input_dim = 37))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 19, activation = 'relu', kernel_initializer = 'uniform'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'uniform'))

    # compile
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return classifier

In [59]:
from sklearn.model_selection import GridSearchCV
classifier = KerasClassifier(build_fn=nn_architecture_3) #dont add batch_size and nb_epochs as we will find them using grid_search
parameters = {'batch_size': [25,32],
             'nb_epoch':[100,500],
             'optimizer':['adam','rmsprop']}

In [60]:
grid_search = GridSearchCV(estimator= classifier, param_grid= parameters, 
                          scoring = 'accuracy',
                          cv = 10)

In [61]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/1
6750/6750 [==============================] - 1s 155us/step - loss: 0.4750 - accuracy: 0.7942
Epoch 1/1
6750/6750 [==============================] - 1s 134us/step - loss: 0.4835 - accuracy: 0.8030
Epoch 1/1
6750/6750 [==============================] - 1s 126us/step - loss: 0.4626 - accuracy: 0.8117
Epoch 1/1
6750/6750 [==============================] - 1s 131us/step - loss: 0.4729 - accuracy: 0.7933
Epoch 1/1
6750/6750 [==============================] - 1s 121us/step - loss: 0.4718 - accuracy: 0.7963
Epoch 1/1
6750/6750 [==============================] - 1s 126us/step - loss: 0.4781 - accuracy: 0.7919
Epoch 1/1
6750/6750 [==============================] - 1s 124us/step - loss: 0.4778 - accuracy: 0.8101
Epoch 1/1
6750/6750 [==============================] - 1s 135us/step - loss: 0.4594 - accuracy: 0.8079
Epoch 1/1
6750/6750 [==============================] - 1s 147us/step - loss: 0.4688 - accuracy: 0.8120
Epoch 1/1
6750/6750 [==============================] - 1s 143us/step - lo

In [63]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [64]:
best_param

{'batch_size': 25, 'nb_epoch': 100, 'optimizer': 'adam'}

In [65]:
best_accuracy

0.8277333333333333